In [335]:
import os
import json
import requests
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
from ydata_profiling import ProfileReport
from urllib.parse import urlparse
import sqlite3
import ast
import numpy as np

In [336]:
# 1_fetch_data.ipynb: Script para obtener datos de la API de TVMaze y guardarlos en formato JSON.

# Crear carpeta json
os.makedirs('./json', exist_ok=True)

def fetch_and_save_data(date):
    """
    Función para obtener datos de la API de TVMaze y guardarlos en formato JSON.
    """
    url = f'http://api.tvmaze.com/schedule/web?date={date}'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        file_path = f'./json/{date}.json'

        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

        print(f'Datos guardados correctamente para la fecha {date}.')
    else:
        print(f'Error al obtener datos para la fecha {date}: {response.status_code}')


# Fechas del 1 de enero al 31 de enero de 2024
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 1, 31)

date_list = [(start_date + timedelta(days=x)).strftime('%Y-%m-%d') for x in range((end_date - start_date).days + 1)]

# Obtener datos para cada día de enero de 2024
for date in date_list:
    fetch_and_save_data(date)

print('Proceso completado.')

Datos guardados correctamente para la fecha 2024-01-01.
Datos guardados correctamente para la fecha 2024-01-02.
Datos guardados correctamente para la fecha 2024-01-03.
Datos guardados correctamente para la fecha 2024-01-04.
Datos guardados correctamente para la fecha 2024-01-05.
Datos guardados correctamente para la fecha 2024-01-06.
Datos guardados correctamente para la fecha 2024-01-07.
Datos guardados correctamente para la fecha 2024-01-08.
Datos guardados correctamente para la fecha 2024-01-09.
Datos guardados correctamente para la fecha 2024-01-10.
Datos guardados correctamente para la fecha 2024-01-11.
Datos guardados correctamente para la fecha 2024-01-12.
Datos guardados correctamente para la fecha 2024-01-13.
Datos guardados correctamente para la fecha 2024-01-14.
Datos guardados correctamente para la fecha 2024-01-15.
Datos guardados correctamente para la fecha 2024-01-16.
Datos guardados correctamente para la fecha 2024-01-17.
Datos guardados correctamente para la fecha 2024

In [337]:
# 2_create_dataframe.ipynb: Script para leer los archivos JSON guardados y crear DataFrames de episodios y shows.

# Crear la carpeta profiling
os.makedirs('./profiling', exist_ok=True)

# Directorio donde están guardados los archivos JSON
json_folder = './json'
dataframes = []

# Leer todos los archivos JSON almacenados
for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        file_path = os.path.join(json_folder, filename)
        
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if isinstance(data, list):  # Verificar que el JSON sea una lista
            df = pd.json_normalize(data)
            dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
if dataframes:
    full_df = pd.concat(dataframes, ignore_index=True)
else:
    raise Exception("No se encontraron archivos JSON válidos en la carpeta.")

# Mostrar información inicial del DataFrame
full_df.info()
full_df.head()

# Crear DataFrame de episodios (episodes_df)
episodes_columns = [
    'id', 'url', 'name', 'season', 'number', 'type', 'airdate', 'airtime', 'airstamp', 
    'runtime', 'rating.average', '_links.self.href', '_links.show.href', '_links.show.name',
    '_embedded.show.id'
]

episodes_df = full_df[episodes_columns].copy()

# Separar información de episodios y renombrar columnas de episodios para consistencia
episodes_df.rename(columns={
    'id': 'episode_id',
    'url': 'episode_url',
    'name': 'episode_name',
    'season': 'season_number',
    'number': 'episode_number',
    'type': 'episode_type',
    'airdate': 'episode_airdate',
    'airtime': 'episode_airtime',
    'airstamp': 'episode_airstamp',
    'runtime': 'episode_runtime',
    'rating.average': 'episode_rating_average',
    '_links.self.href': 'episode_self_href',
    '_links.show.href': 'episode_show_href',
    '_links.show.name': 'episode_show_name',
    '_embedded.show.id': 'show_id'
}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4996 entries, 0 to 4995
Data columns (total 66 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   id                                          4996 non-null   int64  
 1   url                                         4996 non-null   object 
 2   name                                        4996 non-null   object 
 3   season                                      4996 non-null   int64  
 4   number                                      4958 non-null   float64
 5   type                                        4996 non-null   object 
 6   airdate                                     4996 non-null   object 
 7   airtime                                     4996 non-null   object 
 8   airstamp                                    4996 non-null   object 
 9   runtime                                     4437 non-null   float64
 10  image       

C:\Users\kevin\AppData\Local\Temp\ipykernel_2480\1246399053.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat(dataframes, ignore_index=True)


In [338]:
# Crear DataFrame de shows (shows_df)
shows_columns = [col for col in full_df.columns if col not in episodes_columns]
if '_embedded.show.id' not in shows_columns and 'show_id' not in shows_columns:
    shows_columns.append('_embedded.show.id')  # Añadir la columna que necesitas
shows_df = full_df[shows_columns].copy()

In [339]:
# Separar información de shows sin duplicados y renombrar columnas de shows para consistencia
shows_df.rename(columns={
    '_embedded.show.id': 'show_id',
    '_embedded.show.url': 'show_url',
    '_embedded.show.name': 'show_name',
    '_embedded.show.type': 'show_type',
    '_embedded.show.language': 'show_language',
    '_embedded.show.genres': 'show_genres',
    '_embedded.show.status': 'show_status',
    '_embedded.show.runtime': 'show_runtime',
    '_embedded.show.averageRuntime': 'show_average_runtime',
    '_embedded.show.premiered': 'show_premiered',
    '_embedded.show.ended': 'show_ended',
    '_embedded.show.officialSite': 'show_official_site',
    '_embedded.show.schedule.time': 'show_schedule_time',
    '_embedded.show.schedule.days': 'show_schedule_days',
    '_embedded.show.rating.average': 'show_rating_average',
    '_embedded.show.weight': 'show_weight',
    '_embedded.show.network': 'show_network',
    '_embedded.show.webChannel.id': 'show_webChannel_id',
    '_embedded.show.webChannel.name': 'show_webChannel_name',
    '_embedded.show.webChannel.country.name': 'show_webChannel_country_name',
    '_embedded.show.webChannel.country.code': 'show_webChannel_country_code',
    '_embedded.show.webChannel.country.timezone': 'show_webChannel_country_timezone',
    '_embedded.show.webChannel.officialSite': 'show_webChannel_officialSite',
    '_embedded.show.externals.tvrage': 'show_tvrage',
    '_embedded.show.externals.thetvdb': 'show_thetvdb',
    '_embedded.show.externals.imdb': 'show_imdb',
    '_embedded.show.image.medium': 'show_image_medium',
    '_embedded.show.image.original': 'show_image_original',
    '_embedded.show.summary': 'show_summary',
    '_embedded.show.updated': 'show_updated',
    '_embedded.show._links.self.href': 'show_self_href',
    '_embedded.show._links.previousepisode.href': 'show_previousepisode_href',
    '_embedded.show._links.previousepisode.name': 'show_previousepisode_name',
    '_embedded.show._links.nextepisode.href': 'show_nextepisode_href',
    '_embedded.show._links.nextepisode.name': 'show_nextepisode_name',
    '_embedded.show.network.id': 'show_network_id',
    '_embedded.show.network.name': 'show_network_name',
    '_embedded.show.network.country.name': 'show_network_country_name',
    '_embedded.show.network.country.code': 'show_network_country_code',
    '_embedded.show.network.country.timezone': 'show_network_country_timezone',
    '_embedded.show.network.officialSite': 'show_network_official_site',
    '_embedded.show.webChannel': 'show_webChannel',
    '_embedded.show.webChannel.country': 'show_webChannel_country',
    '_embedded.show.dvdCountry.name': 'show_dvdCountry_name',
    '_embedded.show.dvdCountry.code': 'show_dvdCountry_code',
    '_embedded.show.dvdCountry.timezone': 'show_dvdCountry_timezone',
    '_embedded.show.image': 'show_image'
}, inplace=True)

episodes_df.info()
shows_df.info()

# Eliminar duplicados por show_id
shows_df = shows_df.drop_duplicates(subset=['show_id'])

# Generar profiling para `episodes_df`
profile_episodes = ProfileReport(episodes_df, title="Reporte de Profiling - Episodios", explorative=True)
profile_episodes.to_file("./profiling/tvmaze_profiling_episodes.html")

# Generar profiling para `shows_df`
profile_shows = ProfileReport(shows_df, title="Reporte de Profiling - Shows", explorative=True)
profile_shows.to_file("./profiling/tvmaze_profiling_shows.html")

print("Profiling generado para ambos DataFrames y almacenado en la carpeta 'profiling/'.")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4996 entries, 0 to 4995
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   episode_id              4996 non-null   int64  
 1   episode_url             4996 non-null   object 
 2   episode_name            4996 non-null   object 
 3   season_number           4996 non-null   int64  
 4   episode_number          4958 non-null   float64
 5   episode_type            4996 non-null   object 
 6   episode_airdate         4996 non-null   object 
 7   episode_airtime         4996 non-null   object 
 8   episode_airstamp        4996 non-null   object 
 9   episode_runtime         4437 non-null   float64
 10  episode_rating_average  364 non-null    float64
 11  episode_self_href       4996 non-null   object 
 12  episode_show_href       4996 non-null   object 
 13  episode_show_name       4996 non-null   object 
 14  show_id                 4996 non-null   

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

c:\Users\kevin\AppData\Local\Programs\Python\Python312\Lib\site-packages\ydata_profiling\model\pandas\summary_pandas.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.fillna(np.nan)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Profiling generado para ambos DataFrames y almacenado en la carpeta 'profiling/'.


In [340]:
# Verificar cómo se ve la columna actualmente
print(shows_df['show_genres'].head())

0          [Drama, Comedy, Romance]
10    [Drama, Comedy, Supernatural]
12    [Comedy, Adventure, Children]
13                [Comedy, Fantasy]
25                         [Comedy]
Name: show_genres, dtype: object


In [341]:
# 3_data_cleaning.ipynb: Script para limpiar los DataFrames de episodios y shows, eliminando duplicados y valores nulos.

# Crear la carpeta 'data'
data_folder = './data'
os.makedirs(data_folder, exist_ok=True)

# Limpiar episodios
episodes_df = episodes_df.drop_duplicates(subset=['episode_id'])

# Limpiar shows
shows_df = shows_df.drop_duplicates(subset=['show_id'])

# Episodios que deben tener un runtime válido
episodes_df = episodes_df.dropna(subset=['episode_runtime'])

# Rellenar con "Unknown" en lugar de NaN
shows_df['show_genres'].fillna('Unknown', inplace=True)

# Eliminar columnas innecesarias de episodios y shows
shows_df.drop(['image', 'show_image', 'show_network', 'show_webChannel_country'], axis=1, inplace=True)

# Eliminar columnas completamente vacías (0 registros válidos)
episodes_df = episodes_df.dropna(axis=1, how='all')
shows_df = shows_df.dropna(axis=1, how='all')

# 🔄 Guardar DataFrames limpios en archivos Parquet con compresión Snappy
episodes_df.to_parquet(f'{data_folder}/episodes_cleaned.parquet', compression='snappy')
shows_df.to_parquet(f'{data_folder}/shows_cleaned.parquet', compression='snappy')

print("Limpieza de datos completada y archivos guardados con compresión Snappy.")

Limpieza de datos completada y archivos guardados con compresión Snappy.


C:\Users\kevin\AppData\Local\Temp\ipykernel_2480\3393694662.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  shows_df['show_genres'].fillna('Unknown', inplace=True)


In [342]:
# 4_save_to_sql.ipynb: Script para guardar los DataFrames en una base de datos SQLite.

# Crear la carpeta db
db_folder = './db'
os.makedirs(db_folder, exist_ok=True)

# Conectar a la base de datos SQLite
conn = sqlite3.connect(f'{db_folder}/tvmaze_data.db')

# Cargar DataFrames limpios desde archivos Parquet
episodes_df_parquet = pd.read_parquet('./data/episodes_cleaned.parquet')
shows_df_parquet = pd.read_parquet('./data/shows_cleaned.parquet')



# Revisar si alguna celda de show_genres es un ndarray o una lista
invalid_types = shows_df_parquet.apply(lambda x: isinstance(x, (list, tuple, np.ndarray))).sum()
print(f"Cantidad de celdas que son listas o arrays: {invalid_types}")
print(type(shows_df_parquet.iloc[0]))

# Función para convertir cualquier tipo de dato a un formato serializable
def convert_to_json(x):
    if isinstance(x, np.ndarray):
        # Convertir ndarray a lista
        x = x.tolist()
    if isinstance(x, (list, tuple)):
        # Convertir listas y tuplas a JSON strings
        return json.dumps(x)
    return x

# Aplicar la conversión a la columna 'show_genres'
shows_df_parquet = shows_df_parquet.apply(convert_to_json)

# Guardar DataFrames en la base de datos
episodes_df_parquet.to_sql('episodes', conn, if_exists='replace', index=False)
shows_df_parquet.to_sql('shows', conn, if_exists='replace', index=False)

print("DataFrames guardados exitosamente en la base de datos SQLite.")

# Cerrar la conexión
conn.close()

Cantidad de celdas que son listas o arrays: 0
<class 'pandas.core.series.Series'>
DataFrames guardados exitosamente en la base de datos SQLite.


In [345]:
# Cargar el archivo parquet
shows_df_parquet = pd.read_parquet('./data/shows_cleaned.parquet')

# Revisar los primeros registros de la columna 'show_genres'
print(shows_df_parquet['show_genres'].head(10))

# Revisar los tipos de datos presentes en 'show_genres'
type_counts = shows_df_parquet['show_genres'].apply(lambda x: type(x)).value_counts()
print(type_counts)


0                 [Drama, Comedy, Romance]
10           [Drama, Comedy, Supernatural]
12           [Comedy, Adventure, Children]
13                       [Comedy, Fantasy]
25                                [Comedy]
33                                [Comedy]
35                        [Drama, History]
40    [Children, Fantasy, Science-Fiction]
50             [Adventure, Anime, Fantasy]
51     [Action, Adventure, Anime, Fantasy]
Name: show_genres, dtype: object
show_genres
<class 'numpy.ndarray'>    729
Name: count, dtype: int64


In [346]:
def convert_show_genres(value):
    if isinstance(value, str):
        try:
            # Si la columna fue guardada como un string JSON, lo convertimos a lista
            genres = ast.literal_eval(value)
            if isinstance(genres, list):
                return genres
        except:
            pass
    elif isinstance(value, np.ndarray):
        # Si se guardó como numpy array, convertir a lista
        return value.tolist()
    elif isinstance(value, bytes):
        try:
            # Si es un bytes, intentamos decodificarlo
            decoded_value = value.decode('utf-8')
            genres = ast.literal_eval(decoded_value)
            if isinstance(genres, list):
                return genres
        except:
            pass
    return []  # Si falla, devolver una lista vacía

# Aplicar la conversión
shows_df_parquet['show_genres'] = shows_df_parquet['show_genres'].apply(convert_show_genres)

# Verificar el contenido nuevamente
print(shows_df_parquet['show_genres'].head(10))


0                 [Drama, Comedy, Romance]
10           [Drama, Comedy, Supernatural]
12           [Comedy, Adventure, Children]
13                       [Comedy, Fantasy]
25                                [Comedy]
33                                [Comedy]
35                        [Drama, History]
40    [Children, Fantasy, Science-Fiction]
50             [Adventure, Anime, Fantasy]
51     [Action, Adventure, Anime, Fantasy]
Name: show_genres, dtype: object


In [348]:
# 5_analysis.ipynb: Script para realizar análisis sobre los DataFrames de episodios y shows.

# 1. Calcular Runtime Promedio (averageRuntime)
average_runtime = shows_df_parquet['show_average_runtime'].mean()
print(f"Runtime Promedio: {average_runtime} minutos")


# 2. Conteo de shows por género
exploded_genres = shows_df_parquet.explode('show_genres')
genre_counts = exploded_genres['show_genres'].value_counts()
print(f"Conteo de géneros: {genre_counts}")


# 3. Listar dominios únicos del sitio oficial
def extract_domain(url):
    try:
        return urlparse(url).netloc
    except:
        return None

shows_df_parquet['domain'] = shows_df_parquet['show_official_site'].apply(extract_domain)
unique_domains = shows_df_parquet['domain'].dropna().unique()
print("\n Dominios únicos encontrados:")
for domain in unique_domains:
    print(domain)

# Cerrar la conexión
conn.close()


Runtime Promedio: 42.25637181409295 minutos
Conteo de géneros: show_genres
Drama              157
Comedy             123
Romance             73
Adventure           70
Fantasy             69
Action              59
Crime               45
Anime               41
Mystery             31
Thriller            29
History             26
Children            25
Food                22
Sports              20
Travel              18
Music               17
Family              15
Science-Fiction     13
War                 12
Nature              10
Supernatural         8
Medical              8
Horror               7
DIY                  7
Legal                4
Adult                3
Name: count, dtype: int64

 Dominios únicos encontrados:
www.ivi.ru
okko.tv
wink.ru
kion.ru
b''
premier.one
iview.abc.net.au
v.qq.com
v.youku.com
w.mgtv.com
asiapoisk.com
www.bbc.co.uk
www.hotstar.com
smotrim.ru
youtube.com
program.imbc.com
elisaviihde.fi
play.tv2.no
tvn.cjenm.com
www.amazon.co.uk
www.viceland.com
www.wowpres